In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git1 import *

C:\Users\ng947ac\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\jupyter\git\workspace
C:\Users\ng947ac\jupyter\SoSTrades\SoSTrade\pycryptodome-master
C:\Users\ng947ac\jupyter\git\workspace\sos_trades_core


In [3]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# SoSDoeScenario setUp

In [4]:
study_name = 'doe'
ns = f'{study_name}'
sc_name = "SellarDoeScenario"
c_name = "SellarCoupling"
print(sc_name)

SellarDoeScenario


See §2 for Discipmline description

In [5]:
dspace_dict = {'variable': ['x', 'z', 'y_1', 'y_2'],
               'value': [1., [5., 2.], 1., 1.],
               'lower_bnd': [0., [-10., 0.], -100., -100.],
               'upper_bnd': [10., [10., 10.], 100., 100.],
               'enable_variable': [True, True, True, True],
               'activated_elem': [[True], [True, True], [True], [True]]}
#                   'type' : ['float',['float','float'],'float','float']

In [6]:
dspace = pd.DataFrame(dspace_dict)
dspace

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,x,1.0,0.0,10.0,True,[True]
1,z,"[5.0, 2.0]","[-10.0, 0.0]","[10.0, 10.0]",True,"[True, True]"
2,y_1,1.0,-100.0,100.0,True,[True]
3,y_2,1.0,-100.0,100.0,True,[True]


In [7]:
from tempfile import gettempdir
root_dir = gettempdir()
print(root_dir)
dspace.to_csv(root_dir+'\\ds.csv')

C:\Temp


In [8]:
repo = 'sos_trades_core.sos_processes.test'
proc_name = 'test_sellar_doe'

# Doe scenario execution

# 1 Set Doe optim scenario Custom DoE from file

In [9]:
#my_formulation = "DisciplinaryOpt"
my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [10]:
from gemseo.api import (
    configure_logger,
    get_algorithm_options_schema,
    get_available_doe_algorithms,
)
get_available_doe_algorithms()

['CustomDOE',
 'DiagonalDOE',
 'OT_SOBOL',
 'OT_RANDOM',
 'OT_HASELGROVE',
 'OT_REVERSE_HALTON',
 'OT_HALTON',
 'OT_FAURE',
 'OT_MONTE_CARLO',
 'OT_FACTORIAL',
 'OT_COMPOSITE',
 'OT_AXIAL',
 'OT_OPT_LHS',
 'OT_LHS',
 'OT_LHSC',
 'OT_FULLFACT',
 'OT_SOBOL_INDICES',
 'fullfact',
 'ff2n',
 'pbdesign',
 'bbdesign',
 'ccdesign',
 'lhs']

In [11]:
my_doe_algo = "CustomDOE"

In [12]:
print("\n Sellar doe solution check with ",my_formulation ," formulation")


 Sellar doe solution check with  MDF  formulation


In [13]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [14]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	|_ SellarDoeScenario
		|_ Sellar_Problem
		|_ Sellar_2
		|_ Sellar_1


'Nodes representation for Treeview doe\n|_ doe\n\t|_ SellarDoeScenario\n\t\t|_ Sellar_Problem\n\t\t|_ Sellar_2\n\t\t|_ Sellar_1'

In [15]:
exec_eng.dm.get_data('doe.sub_mda_class')['value']

'MDAJacobi'

File '.\\' + 'X_pd.csv' was created from ex_03.1_doe_scenario_lhs_fullfact.ipynb

In [16]:
X_pd = pd.read_csv('.\\' + 'X_pd.csv')

In [17]:
X_pd

,doe.SellarDoeScenario.x,doe.SellarDoeScenario.z,doe.SellarDoeScenario.z.1
0,0.000000,0.000000,1.000000
1,9.379764,1.515949,5.631736
2,8.886448,-1.196271,6.523436
3,3.713714,-1.994758,4.822571
4,0.041702,1.749067,3.617234
...,...,...,...
96,6.661678,5.719822,7.645988
97,2.358656,-6.389327,5.297002
98,3.011475,8.776000,9.837792
99,4.804455,-2.413481,1.131552


Possible algo options:

In [18]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['doe_file', 'samples', 'delimiter', 'comments', 'skiprows', 'eval_jac', 'n_processes', 'max_time', 'wait_time_between_samples']


In [19]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.SellarDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.SellarDoeScenario.algo'] = my_doe_algo
disc_dict[f'{ns}.SellarDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.SellarDoeScenario.formulation'] = my_formulation
disc_dict[f'{ns}.SellarDoeScenario.objective_name'] = 'obj'
#disc_dict[f'{ns}.SellarDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
disc_dict[f'{ns}.SellarDoeScenario.algo_options'] = {
                                #'samples': X_pd,
                                #'samples': X_pd.to_numpy(),
                                'skiprows': 2,
                                'doe_file': '.\\' + 'X_pd.csv'
                                                    }
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0           0.0          10.0             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1         1.0        -100.0         100.0             True   
 3      y_2         1.0        -100.0         100.0             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'skiprows': 2,
  'doe_file': '.\\X_pd.csv'}}

Remark: in fact it is a <b> DoE optim </b> (with 'obj' variable taken as obj function) ! Here dspace variable was used.

In [20]:
exec_eng.get_anonimated_data_dict()

{'<study_ph>.linearization_mode': {'type': 'string',
  'default': 'auto',
  'possible_values': ['auto',
   'direct',
   'adjoint',
   'reverse',
   'finite_differences',
   'complex_step'],
  'numerical': True,
  'io_type': 'in',
  'type_metadata': None,
  'var_name': 'linearization_mode',
  'user_level': 1,
  'range': None,
  'unit': None,
  'description': None,
  'discipline_full_path_list': [],
  'visibility': 'Local',
  'value': 'auto',
  'coupling': False,
  'optional': False,
  'meta_input': False,
  'editable': True,
  'ns_reference': <sos_trades_core.execution_engine.namespace.Namespace at 0x17bf1cbacd0>,
  'model_origin': '01e7d9a6-bda3-41ca-80ce-3ebbcf96041f',
  'disciplines_dependencies': ['01e7d9a6-bda3-41ca-80ce-3ebbcf96041f']},
 '<study_ph>.cache_type': {'type': 'string',
  'default': 'SimpleCache',
  'possible_values': ['SimpleCache', 'HDF5Cache', 'MemoryFullCache'],
  'numerical': True,
  'io_type': 'in',
  'type_metadata': None,
  'var_name': 'cache_type',
  'user_leve

In [21]:
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0           0.0          10.0             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1         1.0        -100.0         100.0             True   
 3      y_2         1.0        -100.0         100.0             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'skiprows': 2,
  'doe_file': '.\\X_pd.csv'}}

In [22]:
exec_eng.dm.set_values_from_dict(disc_dict)

In [23]:
exec_eng.dm.get_data_dict_values()

{'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.max_mda_iter_gs': 5,
 'doe.condition_func': 'true_func',
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.reset_history_each_run': False,
 'doe.linear_solver_MDO': 'gmres',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'gmres',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0

# 2 Display Disciplines I/O variables

In [24]:
sellar1 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_1")[0]

In [25]:
type(sellar1)

sos_trades_core.sos_disciplines.test_discs.sellar.Sellar1

In [26]:
DESC_IN = sellar1.DESC_IN
DESC_OUT = sellar1.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [27]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar


In [28]:
DESC_OUT_df

,type,visibility,namespace
y_1,float,Shared,ns_OptimSellar


In [29]:
sellar2 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_2")[0]

In [30]:
DESC_IN = sellar2.DESC_IN
DESC_OUT = sellar2.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [31]:
DESC_IN_df

,type,visibility,namespace
y_1,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar


In [32]:
DESC_OUT_df

,type,visibility,namespace
y_2,float,Shared,ns_OptimSellar


In [33]:
sellarP = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.Sellar_Problem")[0]

In [34]:
DESC_IN = sellarP.DESC_IN
DESC_OUT = sellarP.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [35]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_1,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar
local_dv,float,NaN,NaN


In [36]:
DESC_OUT_df

,type,visibility,namespace
c_1,array,Shared,ns_OptimSellar
c_2,array,Shared,ns_OptimSellar
obj,array,Shared,ns_OptimSellar


# 3 Provide Discipline inputs

In [37]:
# Sellar inputs
local_dv = 10.
values_dict = {}
values_dict[f'{ns}.{sc_name}.x'] = 1.  
values_dict[f'{ns}.{sc_name}.y_1'] = 1.
values_dict[f'{ns}.{sc_name}.y_2'] = 1.
values_dict[f'{ns}.{sc_name}.z'] = array([1., 1.])
values_dict[f'{ns}.{sc_name}.Sellar_Problem.local_dv'] = local_dv
values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')

,0
doe.SellarDoeScenario.x,1.0
doe.SellarDoeScenario.y_1,1.0
doe.SellarDoeScenario.y_2,1.0
doe.SellarDoeScenario.z,"[1.0, 1.0]"
doe.SellarDoeScenario.Sellar_Problem.local_dv,10.0


In [38]:
exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


# 4 Run Doe scenario

In [39]:
res = exec_eng.execute()
# res is without interest (retourn not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:SellarDoeScenario
   Disciplines: Sellar_Problem Sellar_2 Sellar_1
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSDoeScenario:{'doe_file': '.\\X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 2, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z
INFO:gemseo.algos.driver_lib:DOE sampling:   0%|          | 0/100 [00:00<?, ?it]


my_options
{'doe_file': '.\\X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 2, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}


INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	1.246212e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	2.437255e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	4.667075e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	2.766021e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	5.239659e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	1.659148e-04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	1.776357e-09
INFO:gemseo.algos.driver_lib:DOE sampling:   1%|          | 1/100 [00:00<00:00, 777.26 it/sec, obj=119]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	1.168371e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	2.304065e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	4.477050e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	2.842687e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.765221e

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	4.090707e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.566950e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	4.446188e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	3.140185e-10
INFO:gemseo.algos.driver_lib:DOE sampling:  14%|█▍        | 14/100 [00:01<00:01, 62.14 it/sec, obj=49.3]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	2.827709e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	4.062102e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	7.765717e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	3.870879e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.334660e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	4.081236e-05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	2.041371e-05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	8	0.000000

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.055657e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	2.105808e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	5.024296e-09
INFO:gemseo.algos.driver_lib:DOE sampling:  27%|██▋       | 27/100 [00:03<00:02, 31.89 it/sec, obj=160]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	2.472228e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	3.631425e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	7.080865e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	3.879694e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	2.107093e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	2.962576e-05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	1.256074e-09
INFO:gemseo.algos.driver_lib:DOE sampling:  28%|██▊       | 28/100 [00:03<00:02, 30.22 it/sec, obj=53.4]
INFO:sos_trades_core.execution_engine.gem

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	5.474981e-04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	3.768222e-09
INFO:gemseo.algos.driver_lib:DOE sampling:  40%|████      | 40/100 [00:05<00:03, 17.48 it/sec, obj=22.4]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	2.934943e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	3.924480e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	7.740160e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	4.203443e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	8.402732e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	9.348960e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	4.675107e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	8	1.256074e-09
INFO:gemseo.algos.driver_lib:DOE sampling:  41%|████      | 41/100 [00:05<00:03, 17.10 it/sec, obj=136]
INFO:sos_trades_core.execution_engine.gem

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	7.827780e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	4.466222e-05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	6.280370e-10
INFO:gemseo.algos.driver_lib:DOE sampling:  53%|█████▎    | 53/100 [00:07<00:03, 13.69 it/sec, obj=79.7]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	6.773922e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	6.511646e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	1.260313e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	4.921414e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	1.179889e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	4.604767e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	2.512148e-09
INFO:gemseo.algos.driver_lib:DOE sampling:  54%|█████▍    | 54/100 [00:07<00:03, 13.32 it/sec, obj=177]
INFO:sos_trades_core.execution_engine.gem

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	6.448597e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	6.087678e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	1.207659e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	5.017110e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	2.006058e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	8.038873e-07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	2.303852e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	3.401852e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	6.650006e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	3.972554e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.610540e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	5.162778e-06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	3.140185e-10
INFO:gemseo.algos.driver_

INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	1.900534e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	3.634994e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	2.166799e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	6.443998e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	2.975551e-04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	2.975526e-04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	8	0.000000e+00
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	4.442377e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	5.228456e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	1.003787e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	4.440568e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	2.114750e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	1.449007e-05
INFO:sos_trades_core.exec

INFO:gemseo.algos.driver_lib:DOE sampling:  94%|█████████▍| 94/100 [00:11<00:00,  8.64 it/sec, obj=135]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	2.424596e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	3.726712e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	7.099437e+05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	4	3.641929e+04
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	5	3.981668e+01
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	6	5.916108e-05
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	7	0.000000e+00
INFO:gemseo.algos.driver_lib:DOE sampling:  95%|█████████▌| 95/100 [00:11<00:00,  8.56 it/sec, obj=132]
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	1	3.375268e+07
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	2	4.543358e+06
INFO:sos_trades_core.execution_engine.gemseo_overloads.mda:	3	8.642286e+05
INFO:sos_trades_core.execution_engine.gems

Remark : all options that are None are no kept and so appear with a warning (this is the case of default None options thar are not setted with not None value)

# 5 Display Doe  Scenario results

In [40]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [41]:
type(doe_disc)

sos_trades_core.execution_engine.sos_doe_scenario.SoSDoeScenario

In [42]:
doe_disc.get_disc_full_name()

'doe.SellarDoeScenario'

In [43]:
opt_P = doe_disc.opt_problem
print(opt_P)

Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z


In [44]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

,0
n_samples,100
algo,CustomDOE
design_space,variable value lower_bnd upper...
formulation,MDF
objective_name,obj
differentiation_method,finite_differences
algo_options,"{'doe_file': '.\X_pd.csv', 'delimiter': ',', '..."
parallel_options,"{'parallel': False, 'n_processes': 4, 'use_thr..."
eval_mode,False
eval_jac,False


In [45]:
doe_disc_output = doe_disc.get_sosdisc_outputs()

In [46]:
# doe_disc_output

# 5.1 Table of computed points

In [47]:
XY_pd= doe_disc_output['doe_ds_io']

In [48]:
X_pd = XY_pd['design_parameters']
X_pd

doe.SellarDoeScenario.x doe.SellarDoeScenario.z          
                         0                       0         1
0                 9.379764                1.515949  5.631736
1                 8.886448               -1.196271  6.523436
2                 3.713714               -1.994758  4.822571
3                 0.041702                1.749067  3.617234
4                 6.954955               -9.316161  9.918161
..                     ...                     ...       ...
95                6.661678                5.719822  7.645988
96                2.358656               -6.389327  5.297002
97                3.011475                8.776000  9.837792
98                4.804455               -2.413481  1.131552
99                4.001988                2.954436  9.746571

[100 rows x 3 columns]

In [49]:
Y_pd = XY_pd['functions']
Y_pd

,doe.SellarDoeScenario.obj
,0
0,118.714541
1,110.493389
2,39.894420
3,18.808762
4,169.817929
...,...
95,105.062729
96,68.185137
97,113.215059


# 5.2 Optimum

In [50]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result

{'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'f_opt': 13.915272626018078}

In [51]:
my_optim_result['x_opt']

array([ 1.00983468, -0.65057567,  0.76923226])

In [52]:
my_optim_result['f_opt']

13.915272626018078

# 5.3 From gemseo 

In [53]:
print(opt_P.objective)
print(opt_P.constraints)
print(opt_P.differentiation_method)
print(opt_P.fd_step)

doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
[]
finite_differences
1e-06


In [54]:
dataset = opt_P.export_to_dataset("dataset_name")

In [55]:
XY_pd=dataset.export_to_dataframe()
XY_pd

design_parameters                                    \
   doe.SellarDoeScenario.x doe.SellarDoeScenario.z             
                         0                       0         1   
0                 9.379764                1.515949  5.631736   
1                 8.886448               -1.196271  6.523436   
2                 3.713714               -1.994758  4.822571   
3                 0.041702                1.749067  3.617234   
4                 6.954955               -9.316161  9.918161   
..                     ...                     ...       ...   
95                6.661678                5.719822  7.645988   
96                2.358656               -6.389327  5.297002   
97                3.011475                8.776000  9.837792   
98                4.804455               -2.413481  1.131552   
99                4.001988                2.954436  9.746571   

                   functions  
   doe.SellarDoeScenario.obj  
                           0  
0                 118.714541  
1                 110.493389  
2                  39.894420  
3                  18.808762  
4                 169.817929  
..                       ...  
95                105.062729  
96                 68.185137  
97                113.215059  
98                 45.682186  
99                 54.826284  

[100 rows x 4 columns]

In [56]:
X_pd = XY_pd['design_parameters']
X_pd

doe.SellarDoeScenario.x doe.SellarDoeScenario.z          
                         0                       0         1
0                 9.379764                1.515949  5.631736
1                 8.886448               -1.196271  6.523436
2                 3.713714               -1.994758  4.822571
3                 0.041702                1.749067  3.617234
4                 6.954955               -9.316161  9.918161
..                     ...                     ...       ...
95                6.661678                5.719822  7.645988
96                2.358656               -6.389327  5.297002
97                3.011475                8.776000  9.837792
98                4.804455               -2.413481  1.131552
99                4.001988                2.954436  9.746571

[100 rows x 3 columns]

In [57]:
Y_pd = XY_pd['functions']
Y_pd

,doe.SellarDoeScenario.obj
,0
0,118.714541
1,110.493389
2,39.894420
3,18.808762
4,169.817929
...,...
95,105.062729
96,68.185137
97,113.215059


In [58]:
import numpy as np
min(np.array(Y_pd['doe.SellarDoeScenario.obj']['0']))

13.915272626018078

In [59]:
doe_disc.optimization_result.f_opt

13.915272626018078

In [60]:
doe_disc.optimization_result.x_opt

array([ 1.00983468, -0.65057567,  0.76923226])

In [61]:
doe_disc.print_execution_metrics()

INFO:gemseo.core.scenario:* Scenario Executions statistics *
INFO:gemseo.core.scenario:* Discipline: Sellar_Problem
INFO:gemseo.core.scenario:Executions number: 101
INFO:gemseo.core.scenario:Execution time:  0.033355700000027966 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:* Discipline: Sellar_2
INFO:gemseo.core.scenario:Executions number: 721
INFO:gemseo.core.scenario:Execution time:  0.15390860000003137 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:* Discipline: Sellar_1
INFO:gemseo.core.scenario:Executions number: 721
INFO:gemseo.core.scenario:Execution time:  0.2115649000000115 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:Total number of executions calls 1543
INFO:gemseo.core.scenario:Total number of linearizations 0


In [62]:
doe_disc.get_optimum().__dict__

{'x_0': array([9.37976388, 1.51594904, 5.63173624]),
 'optimizer_name': 'CustomDOE',
 'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'message': None,
 'f_opt': 13.915272626018078,
 'constraints_values': {},
 'constraints_grad': {},
 'status': None,
 'n_obj_call': 100,
 'n_grad_call': None,
 'n_constr_call': None,
 'is_feasible': True}

# 6 Set Doe optim scenario Custom DoE optim from sample

In [63]:
#my_formulation = "DisciplinaryOpt"
my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [64]:
my_doe_algo = "CustomDOE"

In [65]:
X_pd = pd.read_csv('.\\' + 'X_pd.csv')

In [66]:
X_pd

,doe.SellarDoeScenario.x,doe.SellarDoeScenario.z,doe.SellarDoeScenario.z.1
0,0.000000,0.000000,1.000000
1,9.379764,1.515949,5.631736
2,8.886448,-1.196271,6.523436
3,3.713714,-1.994758,4.822571
4,0.041702,1.749067,3.617234
...,...,...,...
96,6.661678,5.719822,7.645988
97,2.358656,-6.389327,5.297002
98,3.011475,8.776000,9.837792
99,4.804455,-2.413481,1.131552


In [67]:
import numpy as np
X_np = np.array(X_pd)
X_np

array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
       [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
       [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
       [ 4.17022005e-02,  1.74906689e+00,  3.61723405e+00],
       [ 6.95495479e+00, -9.31616110e+00,  9.91816129e+00],
       [ 3.99230245e+00, -5.79000931e+00,  6.69501761e+00],
       [ 4.68635419e+00,  6.17857774e+00,  5.93700842e+00],
       [ 9.65724885e+00, -8.16603392e+00,  7.80155333e+00],
       [ 8.58288458e+00, -9.53088785e+00,  5.41567914e+00],
       [ 8.03152448e+00, -5.21931962e+00,  8.46047161e+00],
       [ 8.39186018e+00,  3.38980326e+00,  1.20182883e+00],
       [ 5.92649196e+00, -5.88016462e-01,  2.06944002e+00],
       [ 1.15331653e+00, -4.81449828e+00,  2.96216957e+00],
       [ 1.30233257e-01, -5.91718880e+00,  2.29445948e+00],
       [ 7.03948756e+00,  5.11640284e+00,  6.30659611e+00],
       [ 9.03905478e-01,  8.39476042e+00

In [68]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


Possible algo options: 

In [69]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['doe_file', 'samples', 'delimiter', 'comments', 'skiprows', 'eval_jac', 'n_processes', 'max_time', 'wait_time_between_samples']


In [70]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.SellarDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.SellarDoeScenario.algo'] = my_doe_algo
disc_dict[f'{ns}.SellarDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.SellarDoeScenario.formulation'] = my_formulation
disc_dict[f'{ns}.SellarDoeScenario.objective_name'] = 'obj'
#disc_dict[f'{ns}.SellarDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
disc_dict[f'{ns}.SellarDoeScenario.algo_options'] = {
                                'samples': X_np
                                                    }
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0           0.0          10.0             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1         1.0        -100.0         100.0             True   
 3      y_2         1.0        -100.0         100.0             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'samples': array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
         [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
         [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
         [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
         [ 4.17022005e-0

In [71]:
my_dict = exec_eng.get_anonimated_data_dict()

In [72]:
#pd.DataFrame.from_dict(disc_dict,orient='index')
disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0           0.0          10.0             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 2      y_1         1.0        -100.0         100.0             True   
 3      y_2         1.0        -100.0         100.0             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  
 2         [True]  
 3         [True]  ,
 'doe.SellarDoeScenario.formulation': 'MDF',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'samples': array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
         [ 9.37976388e+00,  1.51594904e+00,  5.63173624e+00],
         [ 8.88644794e+00, -1.19627054e+00,  6.52343621e+00],
         [ 3.71371357e+00, -1.99475780e+00,  4.82257093e+00],
         [ 4.17022005e-0

In [73]:
exec_eng.dm.set_values_from_dict(disc_dict)

In [74]:
exec_eng.dm.get_data_dict_values()

{'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.max_mda_iter_gs': 5,
 'doe.condition_func': 'true_func',
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.reset_history_each_run': False,
 'doe.linear_solver_MDO': 'gmres',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'gmres',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0

In [75]:
# Sellar inputs
local_dv = 10.
values_dict = {}
values_dict[f'{ns}.{sc_name}.x'] = 1.  
values_dict[f'{ns}.{sc_name}.y_1'] = 1.
values_dict[f'{ns}.{sc_name}.y_2'] = 1.
values_dict[f'{ns}.{sc_name}.z'] = array([1., 1.])
values_dict[f'{ns}.{sc_name}.Sellar_Problem.local_dv'] = local_dv
values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')


,0
doe.SellarDoeScenario.x,1.0
doe.SellarDoeScenario.y_1,1.0
doe.SellarDoeScenario.y_2,1.0
doe.SellarDoeScenario.z,"[1.0, 1.0]"
doe.SellarDoeScenario.Sellar_Problem.local_dv,10.0


In [76]:
exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [77]:
exec_eng.dm.get_data_dict_values()

{'doe.linearization_mode': 'auto',
 'doe.cache_type': 'SimpleCache',
 'doe.cache_file_path': None,
 'doe.sub_mda_class': 'MDAJacobi',
 'doe.max_mda_iter': 30,
 'doe.n_processes': 1,
 'doe.chain_linearize': False,
 'doe.tolerance': 1e-06,
 'doe.use_lu_fact': False,
 'doe.warm_start': False,
 'doe.acceleration': 'm2d',
 'doe.warm_start_threshold': -1,
 'doe.n_subcouplings_parallel': 1,
 'doe.max_mda_iter_gs': 5,
 'doe.condition_func': 'true_func',
 'doe.relax_factor': 0.99,
 'doe.epsilon0': 1e-06,
 'doe.reset_history_each_run': False,
 'doe.linear_solver_MDO': 'gmres',
 'doe.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.linear_solver_MDA': 'gmres',
 'doe.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'doe.group_mda_disciplines': False,
 'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'CustomDOE',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x         1.0

Problem : 'doe.SellarDoeScenario.algo_options' has 'doe_file' and not 'samples' has we would like 
It would work if we had algo_default_val = self.default_algo_options_CustomDOE[algo_option] in line 302 of sos_doe_senario.py


In [78]:
res = exec_eng.execute()
# res is without interest (retourn not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:SellarDoeScenario
   Disciplines: Sellar_Problem Sellar_2 Sellar_1
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSDoeScenario:{'doe_file': 'X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 0, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.SellarDoeScenario.obj(doe.SellarDoeScenario.x, doe.SellarDoeScenario.z)
   With respect to: doe.SellarDoeScenario.x, doe.SellarDoeScenario.z
ERROR:gemseo.algos.doe.lib_custom:Failed to load DOE input file: X_pd.csv
ERROR:SoS.EE.SoSDoeScenario:could not convert string to float: 'doe.SellarDoeScenario.x'
Traceback (most recent call last):
  File "C:\Users\ng947ac\jupyter\git\workspace\sos_trades_core\sos_trades_core\execution_engine\sos_discipline.py", line 1

my_options
{'doe_file': 'X_pd.csv', 'delimiter': ',', 'comments': '#', 'skiprows': 0, 'eval_jac': False, 'n_processes': 1, 'max_time': 0, 'wait_time_between_samples': 0.0}


ValueError: could not convert string to float: 'doe.SellarDoeScenario.x'

In [ ]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [ ]:
opt_P = doe_disc.opt_problem
print(opt_P)

In [ ]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

In [ ]:
doe_disc_output = doe_disc.get_sosdisc_outputs()
XY_pd= doe_disc_output['doe_ds_io']

In [ ]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result